## Semantic Kernel: Ramp-Up based on SK's Documentation

To get the latest version of SK Python package, use:

``` bash
pip install --upgrade semantic-kernel
```

## 📒 Notebook 2: AI Services

### 🪜 Step 1: Configure environment

In [1]:
# Import required packages
import os
from IPython.display import Image, display

from semantic_kernel import Kernel
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# Set Azure OpenAI backend variables
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")
AOAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

### 🪜 Step 2: Add AI service

In [7]:
# Initialise kernel
kernel = Kernel()

In [8]:
# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    deployment_name = AOAI_DEPLOYMENT,
    endpoint = AOAI_ENDPOINT,
    api_version = AOAI_API_VERSION,
    service_id = "azure_openai_chat",
)

kernel.add_service(chat_completion)

### 🪜 Step 3: Retrieve AI service

In [9]:
# Retrieve the chat completion service by type
chat_completion_service = kernel.get_service(type=ChatCompletionClientBase)
chat_completion_service.ai_model_type

<OpenAIModelTypes.CHAT: 'chat'>

In [10]:
# Retrieve the chat completion service by id
chat_completion_service = kernel.get_service(service_id="azure_openai_chat")
chat_completion_service.ai_model_type

<OpenAIModelTypes.CHAT: 'chat'>

### 🪜 Step 4: Direct use of Chat Completion services

You can use the chat completion service directly, without adding to the kernel.

In [11]:
# Instantiatte execution settings
execution_settings = OpenAIChatPromptExecutionSettings()

### a) Non-streaming chat completion

In [12]:
# Initialise new chat history
chat_history = ChatHistory()
chat_history.add_user_message("Can you prepare a short (3 paragraphs) story about red pandas?")

In [13]:
# Get non-streaming response
response = await chat_completion_service.get_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

print(response)

In the misty bamboo forests of the Himalayas, a young red panda named Rhea scampered playfully among the leafy branches. With her rust-colored fur and striking black markings, she was a picture of agility and charm. Each day was a new adventure as Rhea explored her favorite tree, expertly balancing on the swaying limbs while munching on tender bamboo shoots. She loved her home, where the sounds of rustling leaves and distant bird songs created a soothing melody, inviting her to discover the secrets of her enchanting habitat.

One sunny afternoon, while Rhea was venturing further than usual, she stumbled upon a hidden grove bathed in golden sunlight. In the center, a tranquil pond shimmered, surrounded by vibrant flowers and lush greenery. As she approached, her curiosity piqued when she spotted a family of colorful butterflies flitting above the water. Completely transfixed, Rhea began to chase them in gleeful circles, spinning and twirling with unbridled joy. For that moment, the worl

### b) Streaming chat completion

In [14]:
# Initialise new chat history
chat_history = ChatHistory()
chat_history.add_user_message("Can you prepare a short (3 paragraphs) story about red pandas?")

In [15]:
# Get streaming response
response = chat_completion_service.get_streaming_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

async for chunk in response:
    print(chunk, end="")

In the heart of a lush, green bamboo forest, a young red panda named Lila lived among the tall trees and vibrant foliage. With her signature reddish-brown fur and big, curious eyes, Lila was known for her playful spirit and adventurous nature. Unlike her more cautious friends, Lila loved to explore the forest, climbing high into the branches and delighting in the rustle of the leaves beneath her paws. She often dreamt of discovering hidden groves and secret spots where the sweetest bamboo grew.

One bright morning, Lila set off on a journey deeper into the forest than she had ever ventured before. As she climbed higher, she stumbled upon a narrow path lined with blooming wildflowers, their sweet scent filling the air. Intrigued, she followed the path until it opened into a sunlit clearing, revealing a serene pond surrounded by the most magnificent bamboo she had ever seen. The sunlight danced on the water's surface, and Lila couldn't resist the temptation to take a refreshing drink and

### 🪜 Step 5: Use rich messages in chat history

In [16]:
# Public image URL for demonstration
hotel_room_image_url = "https://images.pexels.com/photos/164595/pexels-photo-164595.jpeg"

print(f"Anya Sharma's reference image for a hotel room:")
display(Image(url=hotel_room_image_url, width=400)) # Display the image directly in the notebook

Anya Sharma's reference image for a hotel room:


In [17]:
# Initialise new chat history
chat_history = ChatHistory()

chat_history.add_message(
    ChatMessageContent(
        role = AuthorRole.SYSTEM,
        content = "You are a helpful hotel booking assistant. You can assist users in finding and booking rooms, answer questions about hotel amenities, and consider their preferences. Provide personalised suggestions where possible."
    )
)

chat_history.add_message(
    ChatMessageContent(
        role = AuthorRole.USER,
        name = "Anya_Sharma",
        items = [
            TextContent(text="I'm looking to book a room. Can you help me find something similar to what's in this picture?"),
            ImageContent(uri=hotel_room_image_url) # Referencing the image displayed previously
        ]
    )
)

In [18]:
# Simulate function call and result (Tool Message) for user's preferences
chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.ASSISTANT,
        items=[
            FunctionCallContent(
                name = "get_user_preferences_HotelBookingPlugin",
                id = "user_prefs_anya_001",
                arguments = str({"username": "Anya_Sharma"})
            )
        ]
    )
)

# Tool provides the result of the function call (simulated)
chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.TOOL,
        items=[
            FunctionResultContent(
                name = "get_user_preferences_HotelBookingPlugin",
                id = "user_prefs_anya_001",
                result = '{ "loyalty_status": "Diamond", "preferred_amenities": ["king_bed", "balcony", "high_floor", "late_checkout"] }'
            )
        ]
    )
)

In [19]:
# Get assistant's response
assistant_response = await chat_completion_service.get_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

chat_history.add_message(assistant_response)
print(f"Assistant: {assistant_response.content}")

Assistant: Sure, Anya! Based on your loyalty status and preferred amenities, I can help you find hotel rooms that match your style. The room in the picture has a modern aesthetic with elegant décor, which aligns well with upscale hotels.

Here are a few options that feature a similar look and include your preferred amenities:

1. **Luxury Urban Hotel**
   - **Location:** Downtown area
   - **Room Type:** Deluxe King Room
   - **Features:** A modern décor like the one in your picture, a king bed, private balcony, high floor, and option for late checkout.
   - **Amenities:** Spa, rooftop pool, fine dining restaurant.

2. **Contemporary Boutique Hotel**
   - **Location:** Arts district
   - **Room Type:** Executive King Suite
   - **Features:** Stylish design, king bed, balcony with city views, located on a high floor, late checkout available.
   - **Amenities:** Art gallery in the hotel, complimentary breakfast, fitness center.

3. **Chic Resort Hotel**
   - **Location:** Near the beach


In [20]:
# Display the full chat history
print("\n--- Full Rich Chat History ---")
for message in chat_history:
    print(
        f"Role: {message.role}",
        f"Name: {message.name}",
        f"Content: {message.content}",
        f"Items: {message.items}\n"
    )


--- Full Rich Chat History ---
Role: AuthorRole.SYSTEM Name: None Content: You are a helpful hotel booking assistant. You can assist users in finding and booking rooms, answer questions about hotel amenities, and consider their preferences. Provide personalised suggestions where possible. Items: [TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text='You are a helpful hotel booking assistant. You can assist users in finding and booking rooms, answer questions about hotel amenities, and consider their preferences. Provide personalised suggestions where possible.', encoding=None)]

Role: AuthorRole.USER Name: Anya_Sharma Content: I'm looking to book a room. Can you help me find something similar to what's in this picture? Items: [TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text="I'm looking to book a room. Can you help me find something similar to what's in this picture?", encoding=None), ImageContent(inner_conte